In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 87.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 53.0 MB/s eta 0:00:00


In [ ]:
!pip install transformers pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 107.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 21.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import pickle
import json
import os
import re
import logging
from ast import literal_eval
from konlpy.tag import Komoran, Okt
from tqdm.notebook import tqdm, trange
from google.colab import drive
from sklearn.feature_extraction.text import CountVectorizer
from scipy.special import softmax

tqdm.pandas()

In [ ]:
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/Side_project/Brand_Monitoring/")

Mounted at /content/drive


In [ ]:
# Load review data
review = pd.read_csv("main_dictionary.csv")
# Remove rows that have empty col of candiate_ids caused by error or non noun etc.
remove_index = review[(review["candiate_ids"] == "error") | ( review["candiate_ids"] == "{}")].index
review = review.drop(remove_index, axis=0).reset_index(drop=True)
review["candiate_ids"] = review["candiate_ids"].apply(lambda x : literal_eval(x))

In [ ]:
# Load develop, test data & seed words

develop = pd.read_csv("develop.csv", encoding='utf-8-sig')
test = pd.read_csv("test.csv", encoding='utf-8-sig')

with open("mecab_seed.pkl", "rb") as f:
  mecab_seed = pickle.load(f)

with open("komoran_seed.pkl", "rb") as f:
  komoran_seed = pickle.load(f)

with open("okt_seed.pkl", "rb") as f:
  okt_seed = pickle.load(f)

In [ ]:
review_df = review.drop("candiate_ids",axis=1)

In [ ]:
# Additional text cleaning
review_df["review_text"] = review_df["review_text"].progress_apply(lambda x : re.sub('[^A-Za-z0-9가-힣]', ' ', x)).str.strip()
review_df = review_df.dropna(axis=0)

  0%|          | 0/268922 [00:00<?, ?it/s]

In [ ]:
def tokenize_dataset(df):
  # tokenizer
  komoran = Komoran()
  okt = Okt()
  # toknize by 2 tokenizers
  stop_pos = ["MAJ", "IC", "JKS", "JKC", "JKG","JKO", "JKB", "JKV", "JKQ", "JX",
              "JC", "EP", "EF", "EC", "ETN", "ETM", "XSN", "XSV", "XSA", "UNKNOWN", "NF", "NV", "NA", "KoreanParticle"]

  df["komoran_token"] = df["review_text"].progress_apply(lambda x: [word[0] for word in komoran.pos(x) if word[1] not in stop_pos])
  df["okt_token"] = df["review_text"].progress_apply(lambda x: [word[0] for word in okt.pos(x) if word[1] not in stop_pos])

In [ ]:
tokenize_dataset(review_df)

  0%|          | 0/268922 [00:00<?, ?it/s]

  0%|          | 0/268922 [00:00<?, ?it/s]

In [ ]:
def get_seed_words(token_seed):
  aspects = token_seed.keys()
  seed_words = []
  for aspect in aspects:
    temp = token_seed[aspect]
    seed_words += list(temp.keys())
  seed_words = list(set(seed_words))
  return seed_words

In [ ]:
komoran_seed_words = get_seed_words(komoran_seed)
okt_seed_words = get_seed_words(okt_seed)

In [ ]:
review_df = pd.read_csv("train_review.csv", encoding='utf-8-sig')
# review_df.to_csv("train_review.csv", encoding='utf-8-sig', index=False)

# Teacher : Bag_of_Seed_Words

In [ ]:
# Prepare train data (exclude develop, test data)
develop_test_index = np.concatenate([develop["idx"].values, test["idx"].values])
train = review_df.drop(develop_test_index, axis=0)
train = train.dropna(axis=0).reset_index(drop=True)

In [ ]:
develop["okt_token"] = develop["okt_token"].apply(lambda x: literal_eval(x))
aspects = list(develop["aspect1"].unique())
aspects.remove("none")

In [ ]:
class Teacher:
  def __init__(self, token_seed, tokenizer_name, aspects):
    self.token_name = "{}_token".format(tokenizer_name)
    self.token_seed = token_seed
    self.seed_words = get_seed_words(token_seed)
    self.vectorizer = CountVectorizer(decode_error="ignore")
    self.vectorizer.fit([" ".join(self.seed_words)])
    self.aspects = aspects
    self.bag_of_words = None
    self.weight_mat = None

  def pred(self, data):
    # If a token column is a string, not an array change it to array
    train_col =  data[self.token_name]
    if isinstance(train_col.iloc[train_col.first_valid_index()], str):
      data[self.token_name] = data[self.token_name].apply(lambda x: literal_eval(x))
    self.bag_of_words = self.vectorizer.transform(data[self.token_name].apply(lambda x : " ".join(x))).toarray()
    seed_idx_map = self.vectorizer.vocabulary_
    aspect_idx_map = {}
    bag_of_words_pred = np.empty((data.shape[0],1))
    if self.weight_mat is None:
      self.weight_mat = np.ones((data.shape[0], len(aspects)+1))
    for i, aspect in enumerate(aspects):
      aspect_seed = self.token_seed[aspect].keys()
      aspect_idx_map[aspect] = [seed_idx_map[seed] for seed in aspect_seed]
      aspect_seed_weight = self.weight_mat[aspect_idx_map[aspect], i].reshape(1,-1)
      aspect_seed_occur = (self.bag_of_words[:, aspect_idx_map[aspect]] * aspect_seed_weight).sum(axis=1)
      bag_of_words_pred = np.concatenate([bag_of_words_pred, aspect_seed_occur.reshape(-1,1)], axis=1)
    bag_of_words_pred = bag_of_words_pred[:, 1:]
    general_asp_score = (bag_of_words_pred.sum(axis=1) == 0).astype("int64")
    bag_of_words_pred = np.concatenate([bag_of_words_pred, general_asp_score.reshape(-1,1)], axis=1)
    bag_of_words_pred = softmax(bag_of_words_pred, axis=1)

    return bag_of_words_pred
  def update(self, student_pred):
    '''
    weight z_j^k (j-th seed word predictive power of k-th aspect)
    z_j has (1,k) dimension -> z has (N, k) dimension where N is the number of seed words
    '''
    occur_mat = (self.bag_of_words > 0).astype("int64")
    temp_weight = np.matmul(occur_mat.T, student_pred) + np.finfo(float).eps
    self.weight_mat = temp_weight/(temp_weight.sum(axis=1).reshape(-1,1))

# Student : Neural Network

In [ ]:
from transformers import AutoTokenizer, BertModel, BertConfig
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.distributed import DistributedSampler
from transformers.data.metrics import acc_and_f1
from torch import nn
import torch
try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, tokens, labels):
        self.tokens = tokens
        self.labels = labels

    def __getitem__(self, index):
        input_ids = self.tokens["input_ids"][index]
        token_type_ids = self.tokens["token_type_ids"][index]
        attention_mask = self.tokens["attention_mask"][index]
        label = self.labels[index]

        return input_ids, token_type_ids, attention_mask, label, index
    
    def __len__(self):
        return len(self.labels)

In [ ]:
class BertClassifier(nn.Module):
  def __init__(self, model_name, n_labels, dropout=0.0):
    super(BertClassifier, self).__init__()
    config = BertConfig.from_pretrained(model_name)
    self.bert = BertModel.from_pretrained(model_name)
    self.dropout = nn.Dropout(dropout)
    self.linear = nn.Linear(config.hidden_size, n_labels)

  def forward(self, input_id, mask, token_ids):
    _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask, token_type_ids=token_ids, return_dict=False)
    dropout_output = self.dropout(pooled_output)
    output = self.linear(dropout_output)
    return output

In [ ]:
# Setup logging
logger = logging.getLogger(__name__)

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO)

In [ ]:
'''
Arguments
model_name seed, tokenizer_name, aspects, seed_words, lr, weight_decay,batch_size, bootstrap_epoch, scheduler_gamma, epochs, 
'''
class TrainArgs():
  def __init__(self):
    self.model_name = "klue/bert-base"
    self.seed = 777
    self.tokenizer_name = "okt"
    self.aspects = aspects
    self.seed_words = okt_seed
    self.lr = 1e-4
    self.weight_decay = 0
    self.dropout = 0
    self.batch_size = 8
    self.bootstrap_epoch = 0
    self.scheduler_gamma = 0.5
    self.epochs = 8 #default 25
 

In [ ]:
targs = TrainArgs()

In [ ]:
def concat_col_aspects(df):
  return df["aspect1"] + " " +  df["aspect2"] + " " + df["aspect3"]

def evaluate(args, model, val_data, test_data):
  logger.info("***** Running evaluation *****")
  tokenizer = AutoTokenizer.from_pretrained(args.model_name)
  val_token = tokenizer(val_data["review_text"].values.tolist(), 
                          padding=True, truncation=True, return_tensors="pt")
  test_token = tokenizer(test_data["review_text"].values.tolist(), 
                            padding=True, truncation=True, return_tensors="pt")
  
  val_data[["aspect1", "aspect2", "aspect3"]] = val_data[["aspect1", "aspect2", "aspect3"]].fillna("")
  test_data[["aspect1", "aspect2", "aspect3"]] = test_data[["aspect1", "aspect2", "aspect3"]].fillna("")
  label_vectorizer = CountVectorizer()
  eval_aspect = args.aspects + ['none']
  label_vectorizer.fit(eval_aspect)
  val_label_df = val_data.apply(lambda x: concat_col_aspects(x), axis=1)
  test_label_df = test_data.apply(lambda x: concat_col_aspects(x), axis=1)
  val_label = label_vectorizer.transform(val_label_df).toarray()
  test_label = label_vectorizer.transform(test_label_df).toarray()

  val_dataset = CustomDataset(val_token, val_label)
  val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)
  test_dataset = CustomDataset(test_token, test_label)
  test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)

  criterion = nn.CrossEntropyLoss()
  device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
  val_iterator = tqdm(val_loader,desc="Validation Iteration", mininterval=10, ncols=100)
  test_iterator = tqdm(test_loader,desc="Test Iteration", mininterval=10, ncols=100)
  val_loss = 0
  test_loss = 0
  for i, batch in enumerate(val_iterator):
    model.eval()
    batch = tuple(t.to(device) for t in batch)
    inputs = {"input_ids": batch[0], "attention_mask": batch[1],"token_type_ids": batch[2]}
    label = batch[3]
    outputs = model(inputs['input_ids'], inputs['attention_mask'], inputs['token_type_ids'])
    loss = criterion(outputs, label.float())
    val_loss += loss.item()
  val_loss = val_loss/len(val_loader)
  for i, batch in enumerate(test_iterator):
    model.eval()
    batch = tuple(t.to(device) for t in batch)
    inputs = {"input_ids": batch[0], "attention_mask": batch[1],"token_type_ids": batch[2]}
    label = batch[3]
    outputs = model(inputs['input_ids'], inputs['attention_mask'], inputs['token_type_ids'])
    loss = criterion(outputs, label.float())
    test_loss += loss.item()
  test_loss = test_loss/len(test_loader)
  return val_loss, test_loss

In [ ]:
def train_bert(args, train_data, val_data, test_data):
  train_writer = SummaryWriter()
  student_aspects = args.aspects + ["none"]
  # gpu device
  device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
  # sets seeds for numpy, torch and python.random.
  pl.seed_everything(args.seed, workers=True)
  # Get pretrained tokenzier and model(for student classification)
  tokenizer = AutoTokenizer.from_pretrained(args.model_name)
  student_classifier = BertClassifier(args.model_name, n_labels = len(student_aspects), dropout=args.dropout).to(device)
  train_token = tokenizer(train_data["review_text"].values.tolist(), 
                            padding=True, truncation=True, return_tensors="pt")

  # Initialize optimizer, scheduler
  optimizer = torch.optim.Adam(student_classifier.parameters(), lr=args.lr, weight_decay=args.weight_decay)
  ms=args.bootstrap_epoch
  scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[ms, ms+1, ms+2, ms+3], gamma=args.scheduler_gamma)
  # loss function : Cross-entropy
  criterion = nn.CrossEntropyLoss()
  # Initialize teacher classifier
  teacher_classifier = Teacher(args.seed_words, args.tokenizer_name, args.aspects)
  train_iterator = trange(args.epochs, desc="Epoch",mininterval=10, ncols=100)

  # Variables for total logging
  n_steps = 0
 
  logger.info("***** Running training *****")
  for epoch, _ in enumerate(train_iterator):
    # Variables for logging every epoch
    train_loss = 0
    best_val = 10000

    teacher_pred = teacher_classifier.pred(train_data)
    train_dataset = CustomDataset(train_token, teacher_pred)
    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    epoch_iterator = tqdm(train_loader,desc="Iteration", mininterval=10, ncols=100)
    train_iterator.set_description(f"train_epoch: {epoch} train_loss: {train_loss:.4f}")
    for i,batch in enumerate(epoch_iterator):
      student_classifier.train()
      batch = tuple(t.to(device) for t in batch)
      inputs = {"input_ids": batch[0], "attention_mask": batch[1],"token_type_ids": batch[2]}
      label = batch[3]
      outputs = student_classifier(inputs['input_ids'], inputs['attention_mask'], inputs['token_type_ids'])
      loss = criterion(outputs, label)
      loss.backward()
      optimizer.step()
      scheduler.step()
      student_classifier.zero_grad()
      train_loss += loss.item()
      n_steps += 1
      train_writer.add_scalar("Loss/train", loss, n_steps)
      if i % 1000 == 0 :
        print("Iteration {} : {:.4f}".format(i, loss.item()))
    train_loss /= len(train_loader)
    print("Epoch : {} / Train loss : {:.4f}".format(epoch, train_loss))
  # Save student predictions to update teacher classifier
    student_train_pred = np.empty((1,len(student_aspects)))
    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=False)
    with torch.no_grad():
      for i,batch in enumerate(train_loader):
        student_classifier.eval()
        batch = tuple(t.to(device) for t in batch)
        inputs = {"input_ids": batch[0], "attention_mask": batch[1],"token_type_ids": batch[2]}
        outputs = student_classifier(inputs['input_ids'], inputs['attention_mask'], inputs['token_type_ids'])
        student_pred = outputs.detach().cpu().numpy()
        student_train_pred = np.concatenate([student_train_pred, student_pred], axis=0)
    student_train_pred = softmax(student_train_pred[1:,:], axis=1)
    teacher_classifier.update(student_train_pred)
    val_loss, test_loss = evaluate(args, student_classifier, val_data, test_data)
    print("Epoch : {} / Val loss : {:.4f}".format(epoch, val_loss))
    print("Epoch : {} / Test loss : {:.4f}".format(epoch, test_loss))
    train_writer.add_scalar("Loss/val", val_loss, epoch)
    train_writer.add_scalar("Loss/test", test_loss, epoch)
    if best_val > val_loss:
      if not os.path.exists("./models"):
        os.mkdir("./models")
      print("Model saved")
      torch.save(student_classifier.state_dict(), 'models/best_valid_model.pt')
      with open('models/teacher_best_valid_model', 'wb') as f:
        pickle.dump(teacher_classifier, f)
      best_val = val_loss


  train_writer.close()
  return teacher_classifier, student_classifier

In [ ]:
#sample_idx = np.random.choice(range(len(train)), 1000, replace=False)
teacher_model, student_model = train_bert(targs, train, develop, test)

INFO:lightning_fabric.utilities.seed:Global seed set to 777
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch:   0%|                                                                  | 0/8 [00:00<?, ?it/s]

Iteration:   0%|                                                          | 0/33482 [00:00<?, ?it/s]

Iteration 0 : 2.3660
Iteration 1000 : 2.0746
Iteration 2000 : 1.9837
Iteration 3000 : 2.0496
Iteration 4000 : 1.9917
Iteration 5000 : 2.1539
Iteration 6000 : 2.1647
Iteration 7000 : 2.1650
Iteration 8000 : 1.9260
Iteration 9000 : 2.0763
Iteration 10000 : 2.1125
Iteration 11000 : 2.1409
Iteration 12000 : 2.0717
Iteration 13000 : 2.1221
Iteration 14000 : 2.0429
Iteration 15000 : 2.0219
Iteration 16000 : 2.1142
Iteration 17000 : 2.0411
Iteration 18000 : 2.0265
Iteration 19000 : 2.0679
Iteration 20000 : 2.0007
Iteration 21000 : 2.0390
Iteration 22000 : 2.1607
Iteration 23000 : 2.1519
Iteration 24000 : 2.0389
Iteration 25000 : 2.0737
Iteration 26000 : 2.0521
Iteration 27000 : 2.0225
Iteration 28000 : 2.0437
Iteration 29000 : 1.9958
Iteration 30000 : 2.0572
Iteration 31000 : 2.1998
Iteration 32000 : 1.9253
Iteration 33000 : 2.1498
Epoch : 0 / Train loss : 2.0865


Validation Iteration:   0%|                                                  | 0/88 [00:00<?, ?it/s]

...

Exception in thread Thread-24:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.10/dist-packages/tensorboard/summary/writer/event_file_writer.py", line 244, in run
    self._run()
  File "/usr/local/lib/python3.10/dist-packages/tensorboard/summary/writer/event_file_writer.py", line 289, in _run
    self._record_writer.flush()
  File "/usr/local/lib/python3.10/dist-packages/tensorboard/summary/writer/record_writer.py", line 43, in flush
    self._writer.flush()
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/lib/io/file_io.py", line 221, in flush
    self._writable_file.flush()
tensorflow.python.framework.errors_impl.FailedPreconditionError: runs/May19_08-38-17_8e2877915d5f/events.out.tfevents.1684485497.8e2877915d5f.1353.10; Transport endpoint is not connected


Validation Iteration:   0%|                                                  | 0/88 [00:00<?, ?it/s]

Test Iteration:   0%|                                                        | 0/38 [00:00<?, ?it/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Epoch : 1 / Val loss : 2.7898
Epoch : 1 / Test loss : 2.8041
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-55-7c135fe614e5>", line 2, in <cell line: 2>
    teacher_model, student_model = train_bert(targs, train, develop, test)
  File "<ipython-input-48-f9b73c4b3e4d>", line 72, in train_bert
    train_writer.add_scalar("Loss/val", val_loss, epoch)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/tensorboard/writer.py", line 391, in add_scalar
    self._get_file_writer().add_summary(summary, global_step, walltime)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/tensorboard/writer.py", line 113, in add_summary
    self.add_event(event, global_step, walltime)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/tensorboard/writer.py", line 98, in add_event
    self.event_writer.add_event(eve